# Kaggle House Price Prediction – Model Comparison

## Imports and Dependencies

In [127]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import cross_val_predict,KFold
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.linear_model import Ridge, Lasso,ElasticNet
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

## Dataset Loading

In [16]:
train_path=r"D:\Kaggle_house_price\house-prices-advanced-regression-techniques\train.csv"
test_path=r"D:\Kaggle_house_price\house-prices-advanced-regression-techniques\test.csv"

## Training Data Overview

In [17]:
df_train=pd.read_csv(train_path)
df_test=pd.read_csv(test_path)

In [18]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [19]:
df_train.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


## Test Data Overview

In [20]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

In [21]:
df_test.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1459.000000,1459.000000,1232.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1444.000000,1458.000000,...,1458.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000
mean,2190.000000,57.378341,68.580357,9819.161069,6.078821,5.553804,1971.357779,1983.662783,100.709141,439.203704,...,472.768861,93.174777,48.313914,24.243317,1.794380,17.064428,1.744345,58.167923,6.104181,2007.769705
std,421.321334,42.746880,22.376841,4955.517327,1.436812,1.113740,30.390071,21.130467,177.625900,455.268042,...,217.048611,127.744882,68.883364,67.227765,20.207842,56.609763,30.491646,630.806978,2.722432,1.301740
min,1461.000000,20.000000,21.000000,1470.000000,1.000000,1.000000,1879.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,1825.500000,20.000000,58.000000,7391.000000,5.000000,5.000000,1953.000000,1963.000000,0.000000,0.000000,...,318.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000
50%,2190.000000,50.000000,67.000000,9399.000000,6.000000,5.000000,1973.000000,1992.000000,0.000000,350.500000,...,480.000000,0.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,2554.500000,70.000000,80.000000,11517.500000,7.000000,6.000000,2001.000000,2004.000000,164.000000,753.500000,...,576.000000,168.000000,72.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,2919.000000,190.000000,200.000000,56600.000000,10.000000,9.000000,2010.000000,2010.000000,1290.000000,4010.000000,...,1488.000000,1424.000000,742.000000,1012.000000,360.000000,576.000000,800.000000,17000.000000,12.000000,2010.000000


## Target Variable Transformation

In [22]:
y=np.log1p(df_train["SalePrice"])
df_train.drop(columns=["SalePrice"], inplace=True)

## Test Set Identifier Preservation

In [ ]:
test_ids=df_test["Id"]

In [23]:
df=pd.concat([df_train, df_test], axis=0, ignore_index=True)

In [24]:
df.drop(columns=["Id", "Utilities"], inplace=True, errors="ignore")

In [25]:
df["MSSubClass"]=df["MSSubClass"].astype(str)

## Handling Missing Values in Ordinal Categorical Features

In [26]:
ordinal_na_cols = ["BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1","BsmtFinType2",
    "FireplaceQu","GarageQual","GarageCond","GarageFinish","PoolQC","Fence"]
for col in ordinal_na_cols:
    df[col]=df[col].fillna("None")

## Ordinal Encoding Maps Definition and encoding

In [27]:
qual_map={"Ex":5,"Gd":4,"TA":3,"Fa":2,"Po":1,"None":0}
bsmt_exposure_map={"Gd":4,"Av":3,"Mn":2,"No":1,"None":0}
bsmt_fin_map={"GLQ":6,"ALQ":5,"BLQ":4,"Rec":3,"LwQ":2,"Unf":1,"None":0}
garage_finish_map={"Fin":3,"RFn":2,"Unf":1,"None":0}
functional_map={"Typ":7,"Min1":6,"Min2":5,"Mod":4,"Maj1":3,"Maj2":2,"Sev":1,"Sal":0}
fence_map={"GdPrv":4,"MnPrv":3,"GdWo":2,"MnWw":1,"None":0}
lotshape_map={"Reg":3,"IR1":2,"IR2":1,"IR3":0}
landslope_map={"Gtl":2,"Mod":1,"Sev":0}
electrical_map={"SBrkr":4,"FuseA":3,"FuseF":2,"FuseP":1,"Mix":0}
paveddrive_map={"Y":1,"P":0.5,"N":0}

for col in ["ExterQual","ExterCond","BsmtQual","BsmtCond","HeatingQC","KitchenQual","FireplaceQu","GarageQual","GarageCond","PoolQC"]:
    df[col]=df[col].map(qual_map)

df["BsmtExposure"]=df["BsmtExposure"].map(bsmt_exposure_map)
df["BsmtFinType1"]=df["BsmtFinType1"].map(bsmt_fin_map)
df["BsmtFinType2"]=df["BsmtFinType2"].map(bsmt_fin_map)
df["GarageFinish"]=df["GarageFinish"].map(garage_finish_map)
df["Functional"]=df["Functional"].map(functional_map)
df["Fence"]=df["Fence"].map(fence_map)
df["LotShape"]=df["LotShape"].map(lotshape_map)
df["LandSlope"]=df["LandSlope"].map(landslope_map)
df["Electrical"]=df["Electrical"].map(electrical_map)
df["PavedDrive"]=df["PavedDrive"].map(paveddrive_map)

## Neighborhood-Based Imputation for LotFrontage

In [28]:
df["LotFrontage"]=(df.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median())))
df["LotFrontage"]=df["LotFrontage"].fillna(df["LotFrontage"].median())

## One-Hot Encoding: Categorical Feature Selection

In [29]:
onehot_cols = ["MSSubClass","MSZoning","Neighborhood","Street","Alley","LandContour","LotConfig",
               "Condition1","Condition2","BldgType","HouseStyle","Foundation","RoofStyle","RoofMatl",
               "Exterior1st","Exterior2nd","MasVnrType","Heating","GarageType","SaleType","SaleCondition","MiscFeature"]

for col in ["Alley","MasVnrType","GarageType","MiscFeature"]:
    df[col]=df[col].fillna("None")
# Binary Encoding of CentralAir
df["CentralAir"]=df["CentralAir"].map({"Y":1,"N":0})

df=pd.get_dummies(df, columns=onehot_cols, drop_first=True)

## Feature Engineering: Property Age Features

In [30]:
df["HouseAge"]=df["YrSold"]-df["YearBuilt"]
df["RemodAge"]=df["YrSold"]-df["YearRemodAdd"]
df["GarageAge"]=df["YrSold"]-df["GarageYrBlt"]

df.drop(columns=["YearBuilt","YearRemodAdd","GarageYrBlt"], inplace=True)

df["GarageAge"]=df["GarageAge"].fillna(0)
# Zero-Imputation for Area and Count-Based Numerical Features
num_zero_cols=["MasVnrArea","BsmtFinSF1","BsmtFinSF2","BsmtUnfSF","TotalBsmtSF","GarageArea",
               "WoodDeckSF","OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch","PoolArea","MiscVal"]

for col in num_zero_cols:
    df[col]=df[col].fillna(0)

## Log Transformation of Skewed Numerical Features

In [31]:
log_cols = ["LotFrontage","LotArea","MasVnrArea","BsmtFinSF1","BsmtFinSF2","BsmtUnfSF",
            "TotalBsmtSF","1stFlrSF","2ndFlrSF","LowQualFinSF","GrLivArea","GarageArea",
            "WoodDeckSF","OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch","PoolArea","MiscVal"]
for col in log_cols:
    df[col]=np.log1p(df[col])

## Cyclical Encoding of Month Sold

In [32]:
df["MoSold_sin"]=np.sin(2*np.pi*df["MoSold"]/12)
df["MoSold_cos"]=np.cos(2*np.pi*df["MoSold"]/12)
df.drop(columns=["MoSold"], inplace=True)

df["Electrical"] = df["Electrical"].fillna(df["Electrical"].median())

In [34]:
print("X shape:", df.shape)
print("y shape:", y.shape)

# Any NaNs left?
print("Remaining NaNs:", df.isnull().sum().sum())

# Any object columns left?
print("Object columns:", df.select_dtypes(include=["object"]).columns)

X shape: (2919, 213)
y shape: (1460,)
Remaining NaNs: 8
Object columns: Index([], dtype='object')


In [35]:
nan_cols = df.isnull().sum()
nan_cols = nan_cols[nan_cols > 0]
print(nan_cols)

BsmtFullBath    2
BsmtHalfBath    2
KitchenQual     1
Functional      2
GarageCars      1
dtype: int64


## Final Missing Value Cleanup

In [36]:
df["BsmtFullBath"]=df["BsmtFullBath"].fillna(0)
df["BsmtHalfBath"]=df["BsmtHalfBath"].fillna(0)

# Garage capacity (no garage → 0)
df["GarageCars"]=df["GarageCars"].fillna(0)

# Kitchen quality (ordinal → median)
df["KitchenQual"]=df["KitchenQual"].fillna(df["KitchenQual"].median())

# Functional (assume typical)
df["Functional"]=df["Functional"].fillna(functional_map["Typ"])

In [37]:
print("Remaining NaNs:", df.isnull().sum().sum())

Remaining NaNs: 0


In [38]:
X_train=df.iloc[:len(df_train)]
X_test=df.iloc[len(df_train):]

## Scaling of Data

In [84]:
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

## Ridge/L2 regression

In [86]:
ridge = Ridge(alpha=100)
ridge_cv_pred = cross_val_predict(
    ridge,
    X_train_scaled,
    y,
    cv=10
)

ridge_metrics = {
    "R2": r2_score(y, ridge_cv_pred),
    "RMSE": np.sqrt(mean_squared_error(y, ridge_cv_pred)),
    "MAE": mean_absolute_error(y, ridge_cv_pred),
    "MSE": mean_squared_error(y, ridge_cv_pred)
}

print("Ridge CV Metrics")
for k, v in ridge_metrics.items():
    print(f"{k}: {v:.5f}")

Ridge CV Metrics
R2: 0.89441
RMSE: 0.12976
MAE: 0.08604
MSE: 0.01684


## Lasso/L1 regression

In [91]:
lasso=Lasso(alpha=0.001, max_iter=5000)
lasso_cv_pred=cross_val_predict(
    lasso,
    X_train_scaled,
    y,
    cv=10
)
lasso_metrics = {
    "R2": r2_score(y, lasso_cv_pred),
    "RMSE": np.sqrt(mean_squared_error(y, lasso_cv_pred)),
    "MAE": mean_absolute_error(y, lasso_cv_pred),
    "MSE": mean_squared_error(y, lasso_cv_pred)
}
print("\nLasso CV Metrics")
for k, v in lasso_metrics.items():
    print(f"{k}: {v:.5f}")



Lasso CV Metrics
R2: 0.90117
RMSE: 0.12554
MAE: 0.08282
MSE: 0.01576


## XGBoost Regression

In [92]:
xgb=XGBRegressor(
    n_estimators=500,
    learning_rate=0.01,
    max_depth=5,
    subsample=0.75,
    colsample_bytree=0.75,
    random_state=42
)
xgb_cv_pred = cross_val_predict(
    xgb,
    X_train,
    y,
    cv=10
)

xgb_metrics = {
    "R2": r2_score(y, xgb_cv_pred),
    "RMSE": np.sqrt(mean_squared_error(y, xgb_cv_pred)),
    "MAE": mean_absolute_error(y, xgb_cv_pred),
    "MSE": mean_squared_error(y, xgb_cv_pred)
}
print("\nXGBoost CV Metrics")
for k, v in xgb_metrics.items():
    print(f"{k}: {v:.5f}")



XGBoost CV Metrics
R2: 0.89620
RMSE: 0.12865
MAE: 0.08570
MSE: 0.01655


## Net Elastic Regression or L1+L2 combined

In [101]:
enet = ElasticNet(alpha=0.001, l1_ratio=0.5, max_iter=5000)
enet_cv_pred = cross_val_predict(
    enet,
    X_train_scaled,
    y,
    cv=10
)
enet_metrics = {
    "R2": r2_score(y, enet_cv_pred),
    "RMSE": np.sqrt(mean_squared_error(y, enet_cv_pred)),
    "MAE": mean_absolute_error(y, enet_cv_pred),
    "MSE": mean_squared_error(y, enet_cv_pred)
}

print("\nElasticNet CV Metrics")
for k, v in enet_metrics.items():
    print(f"{k}: {v:.5f}")


ElasticNet CV Metrics
R2: 0.90197
RMSE: 0.12502
MAE: 0.08238
MSE: 0.01563


## CatBoost Regressor

In [110]:
cat=CatBoostRegressor(
    iterations=1500,
    learning_rate=0.01,
    depth=6,
    loss_function="RMSE",
    verbose=False,
    random_state=42
)

cat_cv_pred = cross_val_predict(
    cat,
    X_train,
    y,
    cv=10
)

cat_metrics = {
    "R2": r2_score(y, cat_cv_pred),
    "RMSE": np.sqrt(mean_squared_error(y, cat_cv_pred)),
    "MAE": mean_absolute_error(y, cat_cv_pred),
    "MSE": mean_squared_error(y, cat_cv_pred)
}

print("\nCatBoost CV Metrics")
for k, v in cat_metrics.items():
    print(f"{k}: {v:.5f}")



CatBoost CV Metrics
R2: 0.90477
RMSE: 0.12322
MAE: 0.08159
MSE: 0.01518


## Stacking of Net Elastic and Cat Boost

In [126]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)

oof_enet = np.zeros(len(X_train))
oof_cat  = np.zeros(len(X_train))

for tr_idx, val_idx in kf.split(X_train):
    X_tr, X_val = X_train.iloc[tr_idx], X_train.iloc[val_idx]
    y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]

    # ElasticNet
    scaler = RobustScaler()
    X_tr_s = scaler.fit_transform(X_tr)
    X_val_s = scaler.transform(X_val)

    enet = ElasticNet(alpha=0.001, l1_ratio=0.6, max_iter=5000)
    enet.fit(X_tr_s, y_tr)
    oof_enet[val_idx] = enet.predict(X_val_s)

    # CatBoost
    cat = CatBoostRegressor(
        iterations=1500,
        learning_rate=0.01,
        depth=6,
        loss_function="RMSE",
        verbose=False,
        random_state=42
    )
    cat.fit(X_tr, y_tr)
    oof_cat[val_idx] = cat.predict(X_val)

# ============================================================
# META MODEL
# ============================================================
X_meta = np.column_stack([oof_enet, oof_cat])

meta = Ridge(alpha=10)
meta.fit(X_meta, y)

stack_pred = meta.predict(X_meta)

print("\nSTACKED CV METRICS")
print("R2  :", r2_score(y, stack_pred))
print("RMSE:", np.sqrt(mean_squared_error(y, stack_pred)))
print("MAE :", mean_absolute_error(y, stack_pred))

# ============================================================
# FINAL TRAIN + TEST PREDICTION
# ============================================================
scaler = RobustScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s  = scaler.transform(X_test)

enet.fit(X_train_s, y)
enet_test = enet.predict(X_test_s)

cat.fit(X_train, y)
cat_test = cat.predict(X_test)

X_test_meta = np.column_stack([enet_test, cat_test])
final_pred = np.expm1(meta.predict(X_test_meta))

submission = pd.DataFrame({
    "Id": test_ids,
    "SalePrice": final_pred
})

submission.to_csv("submission_stacked.csv", index=False)
print("\nsubmission_stacked.csv created")


STACKED CV METRICS
R2  : 0.9069966783709845
RMSE: 0.12177610620133435
MAE : 0.07854669748985496

submission_stacked.csv created
